# Data

파일 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/캡스톤_FrenchBakery/bakery_sales_top5.csv', parse_dates=['date'])
df = df.sort_values(['article','date'])

# CatBoost (외부 변수 추가)

CatBoost 모델 학습 및 예측

패키지 설치



In [3]:
!pip install catboost
!pip install pycaret-ts-alpha --upgrade

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-1.4.4.tar.gz (4.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pycaret-ts-alpha to determine which version is compatible with other requirements. This could take a while.
  Using cached pycaret_ts_alpha-3.0.0.dev1649017462-py3-none-any.whl.metadata (13 kB)
  Using cached pycaret_ts_alpha-3.0.0.dev1648160839-py3-none-any.whl.metadata (10 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess

### 날씨 + 주말 여부 + 공휴일 여부 추가

In [6]:
import pandas as pd
import requests
from catboost import CatBoostRegressor
import holidays

# 1) 날씨 데이터 조회 함수 (Open-Meteo Archive API 사용)
# 프랑스 파리(48.8566, 2.3522) 기준

def fetch_weather(start_date: str, end_date: str, lat: float = 48.8566, lon: float = 2.3522) -> pd.DataFrame:
    """
    지정 기간(start_date~end_date)의 일별 최고기온(temp_max)과 강수량(precip)을 반환합니다.
    날짜 형식은 'YYYY-MM-DD'.
    """
    url = (
        "https://archive-api.open-meteo.com/v1/era5"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        "&daily=temperature_2m_max,precipitation_sum"
        "&timezone=Europe%2FParis"
    )
    res = requests.get(url)
    data = res.json()["daily"]
    df_w = pd.DataFrame({
        "date": pd.to_datetime(data["time"]),
        "temp_max": data["temperature_2m_max"],
        "precip": data["precipitation_sum"]
    }).set_index("date")
    return df_w

# 2) 판매 데이터 로드 및 정렬
df.sort_values(['article', 'date'], inplace=True)

results = []

# 3) article별 예측 루프
for article in df['article'].unique():
    # 3-1) 수요 시계열 준비
    ts = (
        df[df['article'] == article]
        .set_index('date')['quantity']
        .asfreq('D')
        .fillna(method='ffill')
    )
    start      = ts.index.min()
    train_end  = start + pd.DateOffset(years=1) - pd.Timedelta(days=1)
    test_start = train_end + pd.Timedelta(days=1)
    test_end   = test_start + pd.DateOffset(years=1) - pd.Timedelta(days=1)

    # 3-2) 전체 기간 날씨 데이터 조회
    w_all = fetch_weather(
        start.strftime('%Y-%m-%d'),
        test_end.strftime('%Y-%m-%d')
    )

    # 3-3) Feature engineering: lag, rolling, 날씨, 주말 여부
    feat = ts.to_frame(name='quantity')
    feat['lag_1'] = feat['quantity'].shift(1)
    feat['lag_7'] = feat['quantity'].shift(7)
    feat['roll7'] = feat['quantity'].rolling(7).mean()
    # Open-Meteo 날씨 데이터 합치기
    feat = feat.join(w_all, how='left')
    # 주말(토/일) 플래그
    feat['is_weekend'] = feat.index.weekday >= 5

    # 공휴일 feature 추가
    fr_holidays = holidays.CountryHoliday("FR")
    feat['is_holiday'] = feat.index.to_series().apply(lambda d: int(d in fr_holidays))

    # 결측 보간
    feat.fillna(method='ffill', inplace=True)
    feat.fillna(method='bfill', inplace=True)

    # 3-4) train/test 분리
    train = feat.loc[:train_end].dropna()
    test  = feat.loc[test_start:test_end].dropna()
    if train.empty or test.empty:
        print(f"Skipping {article}: insufficient data")
        continue

    X_train = train.drop(columns='quantity')
    y_train = train['quantity']
    X_test  = test .drop(columns='quantity')
    y_test  = test ['quantity']

    # 3-5) CatBoost 학습 및 예측
    model = CatBoostRegressor(iterations=200, verbose=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 3-6) 결과 저장
    tmp = pd.DataFrame({
        'article':   article,
        'date':      X_test.index,
        'actual':    y_test.values,
        'predicted': y_pred
    })
    results.append(tmp)

# 4) 전체 결과 합치고 CSV 저장
df_results = pd.concat(results, ignore_index=True)
df_results.to_csv('/content/drive/MyDrive/캡스톤_FrenchBakery/article_predictions_final.csv', index=False)
print("Saved → article_predictions_final.csv with weather, weekend, holiday feature")

/tmp/ipython-input-6-3504755756.py:42: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='ffill')
/tmp/ipython-input-6-3504755756.py:70: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat.fillna(method='ffill', inplace=True)
/tmp/ipython-input-6-3504755756.py:71: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat.fillna(method='bfill', inplace=True)
/tmp/ipython-input-6-3504755756.py:42: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='ffill')
/tmp/ipython-input-6-3504755756.py:70: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() 

Saved → article_predictions_final.csv with weather, weekend, holiday feature


/tmp/ipython-input-6-3504755756.py:70: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat.fillna(method='ffill', inplace=True)
/tmp/ipython-input-6-3504755756.py:71: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feat.fillna(method='bfill', inplace=True)


In [7]:
# 5) 오차 계산 (RMSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error
# 전체
rmse_all = mean_squared_error(df_results['actual'], df_results['predicted']) ** 0.5
mae_all = mean_absolute_error(df_results['actual'], df_results['predicted'])
print(f"Overall RMSE: {rmse_all:.2f}, MAE: {mae_all:.2f}")

# article별
errors = df_results.groupby('article').apply(
    lambda g: pd.Series({
        'RMSE': mean_squared_error(g['actual'], g['predicted']) ** 0.5,
        'MAE': mean_absolute_error(g['actual'], g['predicted'])
    })
)
print("Errors by article:")
print(errors)

Overall RMSE: 7.28, MAE: 4.41
Errors by article:
                 RMSE       MAE
article                        
BAGUETTE    10.677805  7.844249
BANETTE     11.349219  8.281946
BANETTINE    1.626496  1.243344
BOULE 200G   2.353337  1.838998
BOULE 400G   3.766661  2.819753


/tmp/ipython-input-7-1600359329.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  errors = df_results.groupby('article').apply(
